<a href="https://colab.research.google.com/github/danielsuassuna04/Neural-Network-Numpy/blob/main/NeuralNetworks_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import copy
import pandas as pd
import math
import time
from IPython.display import clear_output
from sklearn.metrics import accuracy_score,precision_score,f1_score,roc_auc_score

In [8]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [10]:
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 3.9 MB/s eta 0:00:00


In [11]:
dados = pd.read_excel("/content/drive/MyDrive/datasets/Raisin_Dataset.xlsx")
dados
def classe(x):
  if x == "Kecimen":
    return 0
  else:
    return 1
dados["Class"] = dados["Class"].apply(classe)

In [12]:
dados_novo = pd.read_csv("/content/drive/MyDrive/datasets/mushroom_cleaned.csv")
dados_novo

,cap-diameter,cap-shape,gill-attachment,gill-color,stem-height,stem-width,stem-color,season,class
0,1372,2,2,10,3.807467,1545,11,1.804273,1
1,1461,2,2,10,3.807467,1557,11,1.804273,1
2,1371,2,2,10,3.612496,1566,11,1.804273,1
3,1261,6,2,10,3.787572,1566,11,1.804273,1
4,1305,6,2,10,3.711971,1464,11,0.943195,1
...,...,...,...,...,...,...,...,...,...
54030,73,5,3,2,0.887740,569,12,0.943195,1
54031,82,2,3,2,1.186164,490,12,0.943195,1
54032,82,5,3,2,0.915593,584,12,0.888450,1
54033,79,2,3,2,1.034963,491,12,0.888450,1


In [13]:
X = dados_novo.drop("class",axis=1).to_numpy()
Y = dados_novo["class"].to_numpy()

In [14]:
class DeepNetwork():
  def __init__(self):
    return
  def __initialize_parameters_deep(self,layers_dims):
    np.random.seed(3)
    parameters = {}
    L = len(layers_dims)
    for l in range(1,L):
      parameters["W" + str(l)] = np.random.randn(layers_dims[l], layers_dims[l-1]) * np.sqrt(2/layers_dims[l-1])
      parameters["b" + str(l)] = np.zeros((layers_dims[l], 1))
    return parameters
  def __sigmoid(self,Z):
    A = 1/(1+np.exp(-Z))
    cache = Z
    return A, cache
  def __sigmoid_backward(self,dA, cache):
    Z = cache
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    return dZ
  def __relu_backward(self,dA, cache):
    Z = cache
    dZ = dA * np.where(Z > 0, 1, 0)
    return dZ
  def __linear_forward(self,A, W, b):
    Z = np.dot(W,A) + b
    cache = (A, W, b)
    return Z, cache
  def __relu(self,Z):
    A = np.maximum(0,Z)
    cache = Z
    return A, cache
  def __linear_activation_forward(self,A_prev,W,b,activation):
    if activation == "sigmoid":
      Z,linear_cache = self.__linear_forward(A_prev,W,b)
      A,activation_cache = self.__sigmoid(Z)
    elif activation == "relu":
      Z,linear_cache = self.__linear_forward(A_prev,W,b)
      A,activation_cache = self.__relu(Z)
    cache = (linear_cache, activation_cache)
    return A, cache
  def __L_model_forward(self,X, parameters,dropaut=False):
    caches = []
    A = X
    L = len(parameters) // 2
    for l in range(1, L):
      A_prev = A
      if dropaut == False:
        A,cache = self.__linear_activation_forward(A_prev, parameters[f"W{l}"], parameters[f"b{l}"], "relu")
      elif dropaut == True:
        A,cache = self.__linear_activation_forward(A_prev, parameters[f"W{l}"], parameters[f"b{l}"], "relu")
        drop = np.random.rand(A.shape[0],A.shape[1])
        drop = drop < 0.8
        A = np.multiply(A,drop)
      caches.append(cache)
    if dropaut == False:
      AL,cache = self.__linear_activation_forward(A,parameters[f"W{L}"],parameters[f"b{L}"],activation="sigmoid")
    elif dropaut == True:
      AL,cache = self.__linear_activation_forward(A,parameters[f"W{L}"],parameters[f"b{L}"],activation="sigmoid")
      drop = np.random.rand(AL.shape[0],AL) < 0.8
      A = np.multiply(AL,drop)
    caches.append(cache)
    return AL, caches
  def __compute_cost(self,AL, Y):
    m = Y.shape[0]
    cost = -np.sum((Y*np.log(AL) + (1-Y)*np.log(1-AL)))/m
    cost = np.squeeze(cost)
    return cost
  def __linear_backward(self,dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]
    dW = np.dot(dZ,A_prev.T)/m
    db = np.sum(dZ,axis=1,keepdims=True)/m
    dA_prev = np.dot(W.T,dZ)
    return dA_prev, dW, db
  def __linear_activation_backward(self,dA, cache, activation):
    linear_cache, activation_cache = cache
    if activation == "relu":
      dZ = self.__relu_backward(dA, activation_cache)
      dA_prev, dW, db = self.__linear_backward(dZ, linear_cache)
    if activation == "sigmoid":
      dZ = self.__sigmoid_backward(dA, activation_cache)
      dA_prev, dW, db = self.__linear_backward(dZ, linear_cache)
    return dA_prev, dW, db
  def __L_model_backward(self,AL, Y, caches):
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    current_cache = caches[L-1]
    dA_prev_temp, dW_temp, db_temp = self.__linear_activation_backward(dAL,current_cache,"sigmoid")

    grads["dA" + str(L-1)] = dA_prev_temp
    grads["dW" + str(L)] = dW_temp
    grads["db" + str(L)] = db_temp
    for l in reversed(range(L-1)):
      current_cache = caches[l]
      dA_prev_temp, dW_temp, db_temp = self.__linear_activation_backward(dA_prev_temp,current_cache,"relu")
      grads["dA" + str(l)] = dA_prev_temp
      grads["dW" + str(l+1)] = dW_temp
      grads["db" + str(l+1)] = db_temp
    return grads
  def __update_parameters(self,params, grads, learning_rate,optimizer,beta1,beta2):
    parameters = copy.deepcopy(params)
    L = len(parameters) // 2
    Vdw = {}
    Sdw = {}
    if optimizer == "SGD":
        for l in range(L):
            parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate*np.array(grads["dW" + str(l+1)])
            parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate*grads["db" + str(l+1)]
    elif optimizer == "momentum":
        for l in range(L):
            Vdw["VdW" + str(l+1)] = np.zeros(parameters["W" + str(l+1)].shape)
            Vdw["Vdb" + str(l+1)] = np.zeros(parameters["b" + str(l+1)].shape)
        for l in range(L):
            Vdw["VdW" + str(l+1)] = beta1*Vdw["VdW" + str(l+1)] + (1-beta1)*grads["dW" + str(l+1)]
            Vdw["Vdb" + str(l+1)] = beta1*Vdw["Vdb" + str(l+1)] + (1-beta1)*grads["db" + str(l+1)]
            parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate*Vdw["VdW" + str(l+1)]
            parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate*Vdw["Vdb" + str(l+1)]
    elif optimizer == "RMS":
        for l in range(L):
            Vdw["VdW" + str(l+1)] = np.zeros(parameters["W" + str(l+1)].shape)
            Vdw["Vdb" + str(l+1)] = np.zeros(parameters["b" + str(l+1)].shape)
        for l in range(L):
            Vdw["VdW" + str(l+1)] = beta2*Vdw["VdW" + str(l+1)] + (1-beta2)*np.power(grads["dW" + str(l+1)],2)
            Vdw["Vdb" + str(l+1)] = beta2*Vdw["Vdb" + str(l+1)] + (1-beta2)*np.power(grads["db" + str(l+1)],2)
            parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate*(grads["dW" + str(l+1)]/np.sqrt(Vdw["VdW" + str(l+1)]+0.000001))
            parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate*(grads["db" + str(l+1)]/np.sqrt(Vdw["Vdb" + str(l+1)]+0.000001))
    elif optimizer == "Adam":
        for l in range(L):
            Vdw["VdW" + str(l+1)] = np.zeros(parameters["W" + str(l+1)].shape)
            Vdw["Vdb" + str(l+1)] = np.zeros(parameters["b" + str(l+1)].shape)
            Sdw["SdW" + str(l+1)] = np.zeros(parameters["W" + str(l+1)].shape)
            Sdw["Sdb" + str(l+1)] = np.zeros(parameters["b" + str(l+1)].shape)
        for l in range(L):
            Sdw["SdW" + str(l+1)] = beta2*Sdw["SdW" + str(l+1)] + (1-beta2)*np.power(grads["dW" + str(l+1)],2)
            Sdw["Sdb" + str(l+1)] = beta2*Sdw["Sdb" + str(l+1)] + (1-beta2)*np.power(grads["db" + str(l+1)],2)
            Vdw["VdW" + str(l+1)] = beta1*Vdw["VdW" + str(l+1)] + (1-beta1)*grads["dW" + str(l+1)]
            Vdw["Vdb" + str(l+1)] = beta1*Vdw["Vdb" + str(l+1)] + (1-beta1)*grads["db" + str(l+1)]
            Sdw["SdW" + str(l+1)] = Sdw["SdW" + str(l+1)]/(1-(beta2**(l+1)))
            Vdw["VdW" + str(l+1)] = Vdw["VdW" + str(l+1)]/(1-(beta1**(l+1)))
            Sdw["Sdb" + str(l+1)] = Sdw["Sdb" + str(l+1)]/(1-(beta2**(l+1)))
            Vdw["Vdb" + str(l+1)] = Vdw["Vdb" + str(l+1)]/(1-(beta1**(l+1)))
            parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate*(Vdw["VdW" + str(l+1)]/np.sqrt(Sdw["SdW" + str(l+1)] + 0.00001))
            parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate*(Vdw["Vdb" + str(l+1)]/np.sqrt(Sdw["Sdb" + str(l+1)] + 0.00001))
    return parameters
  def fit(self,X, Y, layers_dims, learning_rate = 0.01,beta1=0.9,beta2=0.999, num_iterations = 3000,decay=False,decay_rate=1.0, print_cost=False,dropaut=False,optimizer="SGD"):
    np.random.seed(1)
    costs = 10
    parameters = self.__initialize_parameters_deep(layers_dims)
    learning_rate_0 = learning_rate
    if decay == False:
        for i in range(0, num_iterations):
            AL,caches = self.__L_model_forward(X, parameters)
            cost = self.__compute_cost(AL,Y)
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
            print(f"Accuracy: {accuracy_score(Y,(AL>=0.5).reshape(AL.shape[1],))*100}")
            grads = self.__L_model_backward(AL, Y, caches)
            parameters = self.__update_parameters(parameters, grads, learning_rate,optimizer=optimizer,beta1=beta1,beta2=beta2)
            clear_output(wait=True)

    else:
        for i in range(0, num_iterations):
            learning_rate = (1/(1+(decay_rate*i)))*learning_rate_0
            AL,caches = self.__L_model_forward(X, parameters)
            cost = self.__compute_cost(AL,Y)
            print("Cost after iteration {}: {}".format(i+1, np.squeeze(cost)))
            print(f"Accuracy: {accuracy_score(Y,(AL>=0.5).reshape(AL.shape[1],))*100}")
            grads = self.__L_model_backward(AL, Y, caches)
            parameters = self.__update_parameters(parameters, grads, learning_rate,optimizer=optimizer,beta1=beta1,beta2=beta2)
            clear_output(wait=True)
            costs = cost
    self.parameters = parameters
    return costs

  def predict(self,X):
    AL,caches = self.__L_model_forward(X, self.parameters)
    return AL

In [15]:
teste = DeepNetwork()

In [16]:
def normalization(X):
  mean = np.sum(X,axis=0)/X.shape[0]
  variance = np.sum(X**2,axis=0)/X.shape[0]
  X_novo = X - mean
  X_real = X_novo/np.sqrt(variance)
  return X_real
X_real = normalization(X)

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_real, Y, test_size=0.2, random_state=42)


In [19]:
cost = teste.fit(X_train.T,
                 Y_train,
                (X.shape[1],500,500,500,500,500,1),
                 beta1=0.97131212485,
                 decay=True,
                 decay_rate=0.01,
                 num_iterations=1000,
                 dropaut=False,
                 optimizer="Adam",
                 learning_rate=0.01)

Cost after iteration 1000: 0.029287925918454548
Accuracy: 99.11168686962154


In [20]:
AL = teste.predict(X_test.T)

In [21]:
from sklearn.metrics import roc_auc_score,accuracy_score,precision_score,f1_score
print("metricas nos dados de teste: ")
print("ROC:",end=" ")
print(roc_auc_score(Y_test,(AL>=0.5).reshape(AL.shape[1],))*100)
print("ACC:",end=" ")
print(accuracy_score(Y_test,(AL>=0.5).reshape(AL.shape[1],))*100)
print("PRE:",end=" ")
print(precision_score(Y_test,(AL>=0.5).reshape(AL.shape[1],))*100)
print("F1: ",end=" ")
print(f1_score(Y_test,(AL>=0.5).reshape(AL.shape[1],))*100)

metricas nos dados de teste: 
ROC: 98.97012884411515
ACC: 98.98214120477469
PRE: 99.03422568620806
F1:  99.0677966101695
